In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm as tqdm

In [ ]:
第一步:准备好所有数据
第二部:建模测试

In [ ]:
2 根据上一天的特征预测下一天的流量,end2end
f1:上一天此刻的流量
f2:上一天此刻(n时刻(半小时/附近三小时)范围内的流量)
f3:上一天该站总流量
f5:上一天该线路总流量
f6:上一天该线路该时刻总流量
f7:上一天该节点附近节点流入总流量
f8:上一天该节点附近节点该时刻流入总流量

f4:周几?
f9:时刻
f12:第几周

f10:上一天该时刻数据的可信度
f25:上一天该时刻最多进站站台
f26:上一天该时刻进站最多目标出站站台(f23的max和std)
    
f11:该站台进站口个数+出站口个数
f13:站台的邻居个数
    
f14:上一天乘客该时刻刷卡类型
f15:上一天乘客该时间段刷卡类型
f16:上一天乘客当天刷卡类型(刷卡类型归一化)
    
f21:上一天非周期用户的数量(一天只有两次刷卡)
f22:上一天该站台最常见出站台
f23:上一天该站台全部站出站台人数
f24:上一天该站台全部站进站台人数
分时刻重复上述特征
    
# 进出站交互特征
f17:上一天该时刻进出站总人数
f18:上一天该时间段进出站总人数
f19:上一天该站总进出流量

f20:上一天天气情况

In [17]:
# 统计进出站每天的变化情况
def get_daily_activity(i):
    if i<10:
        s = '0'+str(i)
    else:
        s = str(i)
    day_activ = pd.read_csv('data/Metro_train/record_2019-01-%s.csv'%s)
    day_activ['time'] = pd.to_datetime(day_activ.time)
    return day_activ

In [18]:
day_activ = get_daily_activity(7)

In [19]:
day_activ['minute'] = (day_activ['time'].dt.minute+day_activ['time'].dt.hour*60)//10

,time,lineID,stationID,deviceID,status,userID,payType
0,2019-01-07 00:00:14,B,15,785,0,Dd8e03b983df853eb2bff5b38834beb8e,3
1,2019-01-07 00:00:48,B,5,222,1,De2cd8aad3d8c9315f42f55fe330b0cd1,3
2,2019-01-07 00:00:51,B,5,223,1,D045a19c9d41f734ee6ab1995666bf066,3
3,2019-01-07 00:01:02,C,34,1644,0,Bf9a5d3f0e2eaa5bd08f521f20258cce6,1
4,2019-01-07 00:01:07,C,59,2783,0,D3b81b3d1400d7afadad36bf235cbb406,3
5,2019-01-07 00:01:10,C,34,1643,0,B1117ddc7ada9383fa0e206dd8779e2df,1
6,2019-01-07 00:01:15,B,10,541,0,Dea7be7c1371600e427317ba3fcc42086,3
7,2019-01-07 00:01:16,C,34,1629,0,B2e579fa0480062d568546967f624aaab,1
8,2019-01-07 00:01:17,B,10,541,0,D61f5937bd306e6f1f24ad38703250585,3
9,2019-01-07 00:01:18,C,34,1628,0,B8c460a03977c959e441662bef0bce774,1


In [3]:
connect = roadmap.sum()
connect[connect>2]

5     4
9     3
10    4
11    3
15    3
20    3
46    4
50    3
51    4
52    3
dtype: int64

In [5]:
roadmap

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,1,0,1,0,0,0,...,0,0,0,1,1,0,0,0,0,0
6,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
